In [2]:
from detr_config import Config
from detr_dataset import DataGenerator
from detr_generator import DETRGenerator
from detr_trainer import get_trainer
from detr_file_manager import FileManager
from detr_loader import DETRLoader

import torch

Config.set_local_settings()

# DETR

In [3]:
hyperparameters = ('DETR', 100, 6)

detr_generator = DETRGenerator(*hyperparameters)

image_processor = detr_generator.get_image_processor()

model = detr_generator.get_model()

Model Name:  model=DETR_queries=100_layers=6


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


AttributeError: 'DETRGenerator' object has no attribute 'get_model'

# Deformable DETR

In [3]:
hyperparameters = ('D-DETR', 100, 6)

detr_generator = DETRGenerator(*hyperparameters)

image_processor = detr_generator.get_image_processor()

model = detr_generator.get_model()

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Model Name:  model=D-DETR_queries=100_layers=6


Some weights of DeformableDetrForObjectDetection were not initialized from the model checkpoint at SenseTime/deformable-detr and are newly initialized because the shapes did not match:
- class_embed.0.bias: found shape torch.Size([91]) in the checkpoint and torch.Size([1]) in the model instantiated
- class_embed.0.weight: found shape torch.Size([91, 256]) in the checkpoint and torch.Size([1, 256]) in the model instantiated
- model.query_position_embeddings.weight: found shape torch.Size([300, 512]) in the checkpoint and torch.Size([100, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model_name = detr_generator.get_model_name()
file_manager = FileManager(model_name)
data_generator = DataGenerator(image_processor, file_manager)

file_manager.set_validation_fold_setup(1)

train_dataset, train_loader = data_generator.get_train_dataset_dataloader()
valid_dataset, valid_loader = data_generator.get_valid_dataset_dataloader()

batch = next(iter(train_loader))
print(batch['pixel_values'].shape)

Loading Annotations from:  InBreast_Coco\fold_1\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\fold_1\valid\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
torch.Size([2, 3, 800, 800])


In [5]:
version = 'version-local'
trainer = get_trainer(version)
trainer.fit(model, train_loader, valid_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory ./lightning_logs\version-local\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name | Type                             | Params
----------------------------------------------------------
0 | detr | DeformableDetrForObjectDetection | 39.9 M
----------------------------------------------------------
39.7 M    Trainable params
222 K     Non-trainable params
39.9 M    Total params
159.777   Total estimated model params size (MB)


<generator object Module.named_parameters at 0x0000027001292940>


Sanity Checking: 0it [00:00, ?it/s]

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
c:

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [42]:
detr_loader = DETRLoader(detr_generator, file_manager)
model = detr_loader.load_best_model()

AttributeError: 'DETRLoader' object has no attribute 'file_manager'

In [37]:
# checkpoint_path = f'lightning_logs/{version}/checkpoints/epoch=0-step=1.ckpt'
# checkpoint = torch.load(checkpoint_path)
# state_dict = checkpoint['state_dict']
# state_dict = {k.replace('detr.', ''):state_dict[k] for k in state_dict.keys()}
# detr.load_state_dict(state_dict)

<All keys matched successfully>

In [29]:
checkpoint['state_dict']

{'level_embed': tensor([[ 0.8823,  1.3164, -0.2032,  ..., -0.0117,  0.2053,  0.0444],
         [-1.8036,  1.0426, -0.5457,  ..., -1.4614, -0.0846,  0.6865],
         [-0.2179, -0.3176,  0.5752,  ...,  0.0912, -0.7046,  0.6208],
         [ 1.0417, -0.7510,  0.4228,  ..., -1.1731, -0.3741, -0.0264]]),
 'backbone.conv_encoder.model.conv1.weight': tensor([[[[ 1.3335e-02,  1.4664e-02, -1.5351e-02,  ..., -4.0896e-02,
            -4.3034e-02, -7.0755e-02],
           [ 4.1205e-03,  5.8477e-03,  1.4948e-02,  ...,  2.2060e-03,
            -2.0912e-02, -3.8517e-02],
           [ 2.2331e-02,  2.3595e-02,  1.6120e-02,  ...,  1.0281e-01,
             6.2641e-02,  5.1977e-02],
           ...,
           [-9.0349e-04,  2.7767e-02, -1.0105e-02,  ..., -1.2722e-01,
            -7.6604e-02,  7.8453e-03],
           [ 3.5894e-03,  4.8006e-02,  6.2051e-02,  ...,  2.4267e-02,
            -3.3662e-02, -1.5709e-02],
           [-8.0029e-02, -3.2238e-02, -1.7808e-02,  ...,  3.5359e-02,
             2.2439e-02,